In [ ]:
# The aim of this module is to create ticker level data for calculation

# File name convention
# R_ = raw extract 
# E_ = Entry files into enrichment layer
# C_ = enriched layer with calculation
# I = insights layer, designed for model baselines
# V_ = validation

In [2]:
import pandas as pd
import numpy as np

In [3]:


I_TIC = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/E_TICK_INPUTS.pkl') 
I_INFO = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/E_ALL_INFO.pkl') 
I_DIV = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/E_ALL_Div.pkl') 
I_PRICE = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/E_ALL_Prices.pkl') 
I_FIN = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/E_ALL_FIN.pkl') 


TIC = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/R_ALL_TICKER.pkl') 
TIC_U = TIC.drop_duplicates(subset=['colummn']).rename(columns={'colummn': 'TickName'})[['TickName']]

del TIC

In [4]:
# Join it back together

TIC_FLAT = TIC_U.merge(I_TIC,how='inner',on='TickName').merge(I_DIV,how='inner',on='TickName').merge(I_INFO,how='inner',on='TickName').merge(I_PRICE,how='inner',on='TickName').merge(I_FIN,how='inner',on='TickName')
# full join when all financials are downloaded
# TIC_FLAT = TIC_U.merge(I_TIC,how='left',on='TickName').merge(I_DIV,how='left',on='TickName').merge(I_INFO,how='left',on='TickName').merge(I_PRICE,how='left',on='TickName').merge(I_FIN,how='left',on='TickName')
# .fillna('Missing')

TIC_FLAT_NAN = TIC_FLAT.isna().sum()

# from datetime import datetime as dt
# dte = dt.now().strftime('%Y%m%d')
# fname = "/Users/joezhou/Downloads/TIC_FLAT_NAN_COUNT{}.xlsx".format(dte)
# TIC_FLAT_NAN.to_excel(fname)

In [11]:
# calculate key features

TIC_FLAT['CALC_CURR_RATIO'] = TIC_FLAT['Total Current Assets']/TIC_FLAT['Total Current Liabilities']
TIC_FLAT['CALC_PE_RATIO'] = TIC_FLAT['Last_Close_price']/(TIC_FLAT['Net Income']/TIC_FLAT['sharesOutstanding'])

TIC_FLAT['CALC_DIV_OPEX_RATE'] = TIC_FLAT['Dividends Paid'] / TIC_FLAT['Total Operating Expenses']
TIC_FLAT['CALC_TAX_RATE'] = TIC_FLAT['Income Tax Expense'] / TIC_FLAT['Income Before Tax']

TIC_FLAT['CALC_TOTAL_DEBT'] = TIC_FLAT['Short Long Term Debt'] + TIC_FLAT['Long Term Debt'] 
TIC_FLAT['CALC_TOTAL_CAPITAL'] = TIC_FLAT['CALC_TOTAL_DEBT'] + TIC_FLAT['Total Stockholder Equity']
TIC_FLAT['CALC_PERC_DEBT'] = TIC_FLAT['CALC_TOTAL_DEBT'] / TIC_FLAT['CALC_TOTAL_CAPITAL']
TIC_FLAT['CALC_PERC_EQTY'] = TIC_FLAT['Total Stockholder Equity'] / TIC_FLAT['CALC_TOTAL_CAPITAL']
TIC_FLAT['CALC_DEBT_EQUITY'] = TIC_FLAT['CALC_TOTAL_DEBT'] / TIC_FLAT['Total Stockholder Equity']


TIC_FLAT['CALC_COST_DEBT'] = TIC_FLAT['Long Term Debt']/TIC_FLAT['Total Assets']




# TIC_FLAT['CALC_WACC']= (TIC_FLAT['CALC_PERC_EQTY']*TIC_FLAT[''])+ (TIC_FLAT['CALC_PERC_DEBT']*TIC_FLAT['CALC_COST_DEBT'])*(1-TIC_FLAT['CALC_TAX_RATE']) 

In [12]:
TIC_FLAT.to_pickle("/Users/joezhou/Downloads/Investment/1 data/C_TICKER_FLAT.pkl")

